In [1]:
import os
import re

import numpy as np

In [2]:
def load_data(is_test=False):
    
    if is_test:
        test_txt = 'input_.txt'
    else:
        test_txt = 'input.txt'
        
    with open(test_txt) as f:
        lines = f.read().strip().split('\n')
    
    data = np.array([np.array(list(line)) for line in lines])
    data_padded = np.pad(data, 1, mode='constant', constant_values=('.'))
    
    return lines, data, data_padded 


def get_adjacents(data_padded, c, r_start, r_end):
    '''
    line[r_start:r_end] is the number. 
    '''
    # number
    # data_padded[c+1, r_start+1:r_end+1]
    adjacents = []
    # above
    adjacents.extend(list(data_padded[c, r_start:r_end+2]))
    # right and left
    adjacents.append(data_padded[c+1, r_start])
    adjacents.append(data_padded[c+1, r_end+1])
    # below
    adjacents.extend(data_padded[c+2, r_start:r_end+2])
    
    return adjacents

In [3]:
lines, data, data_padded = load_data(is_test=False)

part_numbers_all = 0
not_part_numbers = []
for c, line in enumerate(lines):
    numbers = re.findall('\d+', line)
    number_positions = [(m.start(0), m.end(0)) for m in re.finditer('\d+', line)]
    
    if len(numbers) > 0: # if there are numbers in the line.
        for number, number_position in zip(numbers, number_positions):
            # row where the number start/end
            #r_start = line.find(number)
            #r_end = r_start + len(number)
            r_start, r_end = number_position
            
            ## get adjacent cells.
            adjacents = get_adjacents(data_padded, c, r_start, r_end)
            adjacents_ = [x for x in adjacents if x not in ['.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']]
            #adjacents_ = [x for x in adjacents if x not in ['.']]
            if len(adjacents_) > 0:
                #print(f"{number} ({c}, {r_start}-{r_end}): {adjacents_}")
                #print(f"{adjacents}")
                part_numbers_all += int(number)
            else:
                not_part_numbers.append((c, r_start, r_end))
                #print(f"{number} ({c}, {r_start}-{r_end}): {adjacents_}")
print(f"the sum of all of the part numbers in the engine schematic: {part_numbers_all}")

the sum of all of the part numbers in the engine schematic: 550064


In [5]:
## circle non-parts (for debug)
data_ = np.copy(data_padded)
for x in not_part_numbers:
    c, r_start, r_end = x
    adjacents = get_adjacents(data_padded, c, r_start, r_end)
    data_[c, r_start:r_end+2] = '@'
    data_[c+1, r_start] = '@'
    data_[c+1, r_end+1] = '@'
    data_[c+2, r_start:r_end+2] = '@'

# output
n_rows, n_cols = np.shape(data_)
with open("output.txt", "w") as f:
    for r in range(n_rows):
        line = data_[r, :]
        f.write(''.join(line)+'\n')